In [1]:
import os
import sys
from dotenv import load_dotenv
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import pickle
import codecs

scripts_path = os.getcwd()
env_file_path = os.path.join(scripts_path, 'env.txt')
load_dotenv(env_file_path)
main_path = os.getenv("MAIN_PATH")

sys.path.append(main_path)
data_path = os.path.join(main_path, 'data')
models_path = os.path.join(main_path, 'models','DS06G_XXcoefficientiModelli')
models_xgb_path = os.path.join(main_path, 'models','DS06B_XXmodelCorporate')

In [3]:
samples_file = os.path.join(data_path, 'EW_BusinessXai_v2.xlsx')
samples = pd.read_excel(samples_file, sheet_name = 'EW_MODULI_SCORE_CORPORATE')
scores = pd.read_excel(samples_file, sheet_name = 'EW_SCORE_CORPORATE').set_index('ID')
kpi_map = pd.read_excel(os.path.join(data_path, 'mapping descrizioni KPI_v6.1.xlsx'))

kpi_map = kpi_map.rename(columns={'ID':'KPI'})
kpi_map = kpi_map[kpi_map.NOME_COLONNA_OUTPUT=='VALORE'].drop(['TIPOLOGIA_OUTPUT','COL_TO_TRANSPOSE','VALUE','CONSTANT_TO_ADD','TO_DROP','DESCRIZIONE','DATA_INIZIO_VALIDITA','DATA_FINE_VALIDITA'], axis = 1)
kpi_map = kpi_map[(kpi_map.KPI.isin(samples.KPI_CD.unique()))].drop(['NOME_COLONNA_OUTPUT', 'TABELLA'], axis = 1)
kpi_map = samples.loc[:,['MODULO_DS', 'KPI_CD']].drop_duplicates().set_index('KPI_CD').join(kpi_map.set_index('KPI')).reset_index().rename(columns={'KPI_CD':'KPI'})

data = samples.pivot(index = 'ID', columns = 'KPI_CD', values='VALORE_QT')

descriptions = pd.read_excel(os.path.join(data_path, 'logit_weights.xlsx'), sheet_name='weights')
descriptions = descriptions.set_index('Variabile').join(kpi_map.drop('MODULO_DS', axis=1).set_index('CAMPO'), how = 'inner').reset_index().sort_values(['MODELLO','MODULO_DS']).set_index('KPI')
kpi_descriptions = descriptions.loc[:,'Descrizione'].to_dict()
kpi_name = descriptions.loc[descriptions.index.dropna(),'Variabile'].to_dict()
name_kpi = {name:kpi for kpi,name in kpi_name.items()}

integration_cols = ['PD_MEDIA_UFFICIALE_M1_QT','PREVISIONE_WORST_QT']
integration_data = scores.loc[:,integration_cols].rename(columns = {'PD_MEDIA_UFFICIALE_M1_QT':'binned_PD_MEDIA_UFFICIALE_M1','PREVISIONE_WORST_QT':'binned_VAR_PREVISIONE_4_0_base_oss'})

In [4]:
slicer = pd.IndexSlice
parquet_files = os.listdir(models_path)[:-1]
models = pd.concat([pd.read_parquet(os.path.join(models_path, pqt_file)) for pqt_file in parquet_files]).reset_index(drop = True)
models['value_type'] = models.apply(lambda x: x.NAME.split('_')[0], axis = 1)

In [5]:
## CONSIDERO CORPORATE
models_corporate = models[models.MODELLO == 'Corporate']

medie_corporate = models_corporate[models_corporate.value_type == 'media']
medie_corporate['NAME'] = medie_corporate.NAME.str.replace('media_','')
pivot_medie_corporate = medie_corporate.pivot(index = ['MODELLO','MODULO','NAME'], values='VALUE', columns='value_type')

models_features_corporate = models_corporate.drop(medie_corporate.index).set_index(['MODELLO','MODULO','NAME'])

logit_models_corporate = models_features_corporate.join(pivot_medie_corporate, how = 'left').loc[slicer['Corporate',['ANDAMENTALE_INTERNO','BILANCI','CR','INTEGRAZIONE_SCORE_MODULI', 'INTEGRAZIONE_INFO']], :]
logit_models_corporate = logit_models_corporate.droplevel(0)

C:\Users\GiorgioBarazzetta\AppData\Local\Temp\ipykernel_26536\3230101557.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medie_corporate['NAME'] = medie_corporate.NAME.str.replace('media_','')


In [6]:
import shap

class XGBoost:
    def __init__(self, df):
        modelpickle = df['model'].iloc[0]
        model = pickle.loads(codecs.decode(modelpickle.encode(), "base64"))
        self.output_function = model
    
    def predict(self, X):
        pred = self.output_function.predict_proba(X)[:,1]
        pred = pd.Series(pred, index = X.index)
        return pred
    
    def explain(self, X):
        explainer = shap.TreeExplainer(model=self.output_function)
        explanations = explainer.shap_values(X.values)
        explanations = pd.DataFrame(explanations, columns=X.columns, index = X.index)
        return explanations



class Logit:
    def __init__(self, betas, intercept, averages):
        self.betas = betas
        self.intercept = intercept
        self.averages = averages
    
    def predict(self, X):
        log_odds_ratio  = X.transpose().apply(lambda x: (x*self.betas).sum()).dropna() + self.intercept
        # pred = log_odds_ratio.apply(lambda x: 1/(1+np.exp(-1*x)))
        return log_odds_ratio
    
    def explain(self, X):
        explanations = X.transpose().apply(lambda x: (x - self.averages)*self.betas).dropna().transpose()
        return explanations
    
    def predict_proba(self,X):
        log_odds_ratio = self.predict(X)
        pred = log_odds_ratio.apply(lambda x: 1/(1+np.exp(-1*x)))
        return pred

class Binner:
    def __init__(self, table):
        tables_rows = list(table.itertuples(index = False, name = None))
        self.bins_mapper = {pd.IntervalIndex.from_tuples([(x[0], x[1])], closed='left')[0]: x[2] for x in tables_rows}
        self.bins = pd.IntervalIndex.from_tuples([(x[0], x[1]) for x in tables_rows], closed = 'left')
    
    def transform(self, X):
        binned_X = pd.Series(pd.cut(X.values.reshape(-1), self.bins), index = X.index).replace(self.bins_mapper).astype(float)
        return binned_X

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [7]:
transformation_path = os.path.join(main_path, 'models','DS06G_XX0BinnedScoresCorp')
transformation_files = os.listdir(transformation_path)[:-1]
transformation = pd.concat([pd.read_parquet(os.path.join(transformation_path, pqt_file)) for pqt_file in transformation_files]).reset_index(drop = True)
transformation = transformation.rename(columns = {'feature':'MODULO'})
map_score = {'score_bil':'BILANCI', 'score_AI': 'ANDAMENTALE_INTERNO', 'score_cr':'CR', 'score_trans': 'TRANSAZIONALE'}
transformation = transformation.replace(map_score)

In [8]:
contribs = []
pc_contribs = []
preds = []
binned_preds = []
renamed_data = data.rename(columns=kpi_name)
binned_score_map = {'BILANCI': 'binned_score_bil', 
                    'ANDAMENTALE_INTERNO': 'binned_score_AI', 
                    'CR': 'binned_score_cr', 
                    'TRANSAZIONALE':'binned_score_trans' }

for modulo in ['ANDAMENTALE_INTERNO', 'BILANCI', 'CR']:
    intercept = logit_models_corporate.loc[slicer[modulo, 'intercetta'], 'VALUE']
    betas = logit_models_corporate.loc[slicer[modulo], 'VALUE'].drop('intercetta')
    averages = logit_models_corporate.loc[slicer[modulo], 'media'].dropna()
    model = Logit(betas, intercept, averages)

    explanations = model.explain(renamed_data)
    explanations.columns = pd.MultiIndex.from_product([[binned_score_map[modulo]], explanations.columns])
    pc_explanations = (explanations.transpose()/explanations.sum(1)).transpose()

    predictions = model.predict(renamed_data)

    current_transformation = transformation[transformation.MODULO == modulo].iloc[:,1:4]
    binned_predictions = Binner(current_transformation).transform(predictions)

    contribs += [explanations]
    pc_contribs += [pc_explanations]
    preds += [predictions.to_frame(modulo)]
    binned_preds += [binned_predictions.to_frame(binned_score_map[modulo])]



## TRANSAZIONALE
trans_features = kpi_map[kpi_map['MODULO_DS'] == 'Transazionale'].CAMPO.values.tolist()
trans_data = renamed_data.loc[:, trans_features]
files = os.listdir(models_xgb_path)[:-1]
pd_xgb_model = pd.read_parquet(os.path.join(models_xgb_path, files[0]))
xgboost_model = XGBoost(pd_xgb_model)

trans_explanations = xgboost_model.explain(trans_data)
trans_explanations.columns = pd.MultiIndex.from_product([[binned_score_map['TRANSAZIONALE']], trans_explanations.columns])
pc_trans_explanations = (trans_explanations.transpose()/trans_explanations.sum(1)).transpose()

trans_preds = xgboost_model.predict(trans_data)

current_transformation = transformation[transformation.MODULO == 'TRANSAZIONALE'].iloc[:,1:4]
trans_binned_predictions =  Binner(current_transformation).transform(trans_preds)

contribs += [trans_explanations]
pc_contribs += [pc_trans_explanations]
preds += [trans_preds.to_frame('TRANSAZIONALE')]
binned_preds += [trans_binned_predictions.to_frame(binned_score_map['TRANSAZIONALE'])]

contribs = pd.concat(contribs, axis = 1)
pc_contribs = pd.concat(pc_contribs, axis = 1)
preds = pd.concat(preds, axis = 1)
binned_preds = pd.concat(binned_preds, axis = 1)


## INTEGRAZIONE SCORE MODULI
intercept_integrated = logit_models_corporate.loc[slicer['INTEGRAZIONE_SCORE_MODULI', 'intercetta'], 'VALUE']
betas_integrated = logit_models_corporate.loc[slicer['INTEGRAZIONE_SCORE_MODULI'], 'VALUE'].drop('intercetta')
averages_integrated = logit_models_corporate.loc[slicer['INTEGRAZIONE_SCORE_MODULI'], 'media'].dropna()
model_integrated = Logit(betas_integrated, intercept_integrated, averages_integrated)

explanations_integrated = model_integrated.explain(binned_preds)
predictions_integrated = model_integrated.predict(binned_preds).to_frame('score_integrated')

## INTEGRAZIONE INFO
intercept_ew = logit_models_corporate.loc[slicer['INTEGRAZIONE_INFO', 'intercetta'], 'VALUE']
betas_ew = logit_models_corporate.loc[slicer['INTEGRAZIONE_INFO'], 'VALUE'].drop('intercetta')
averages_ew = logit_models_corporate.loc[slicer['INTEGRAZIONE_INFO'], 'media'].dropna()
model_ew = Logit(betas_ew, intercept_ew, averages_ew)

ew_dataset = pd.concat([predictions_integrated, integration_data],axis = 1)

explanations_ew = model_ew.explain(ew_dataset)
predictions_ew = model_ew.predict(ew_dataset).to_frame('score_ew')
proba_ew = model_ew.predict_proba(ew_dataset)

### BACKWARD CONTRIBUTIONS
scores_to_ew_contributions = betas_ew.loc['score_integrated']*explanations_integrated
features_to_scores = pc_contribs.apply(lambda x: x.transpose()*scores_to_ew_contributions.loc[:,x.name[0]].transpose())
explanations_integration = explanations_ew.loc[:,integration_data.columns]
explanations_integration.columns = pd.MultiIndex.from_product([['INTEGRAZIONE_INFO'], explanations_integration.columns])
features_to_scores = pd.concat([features_to_scores, explanations_integration], axis = 1)

In [ ]:
### save
explanations_ew.columns = pd.MultiIndex.from_product([['INTEGRAZIONE_INFO'],explanations_ew.columns])
explanations_integrated.columns = pd.MultiIndex.from_product([['INTEGRAZIONE_SCORE_MODULI'],explanations_integrated.columns])
expl = pd.concat([
    contribs.melt(ignore_index=False, var_name=['modulo','variable_name']),
    explanations_ew.melt(ignore_index=False, var_name=['modulo','variable_name']),
    explanations_integrated.melt(ignore_index=False, var_name=['modulo','variable_name'])
])
# writer = pd.ExcelWriter(os.path.join(data_path,'example.xlsx'))
# expl.to_excel(writer, sheet_name = 'contribs')


### INTERPRETAZIONE RISULTATI

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import openai

base_llm = ChatOpenAI(model="gpt-4", temperature = 0, openai_api_key= os.environ.get('OPENAI_API_KEY'))

The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.


In [62]:
features_to_scores.transpose().groupby(level = 0).sum()

ID,1,2
INTEGRAZIONE_INFO,0.234379,0.234379
binned_score_AI,1.217400,1.217400
binned_score_bil,0.237929,0.237929
binned_score_cr,0.818009,0.818009
binned_score_trans,0.265731,0.265731


In [58]:
scores_to_ew_contributions

,binned_score_AI,binned_score_bil,binned_score_cr,binned_score_trans
ID,,,,
1,1.2174,0.237929,0.818009,0.265731
2,1.2174,0.237929,0.818009,0.265731


NAME
score_integrated                      0.925160
binned_PD_MEDIA_UFFICIALE_M1         -0.193576
binned_VAR_PREVISIONE_4_0_base_oss   -0.956618
Name: VALUE, dtype: float64

In [65]:
variables_desc = pd.read_excel(os.path.join(data_path, 'logit_weights.xlsx'), sheet_name='weights')
desc = variables_desc[(variables_desc.MODULO_DS!='Integrazione') & (-variables_desc.Variabile.isin(['intercept','Intercept','score_integrated']))].loc[:,['Variabile','Descrizione']].set_index('Variabile').join(features_to_scores.round(3).loc[1].droplevel(0).to_frame('SHAP_values'))
mkd_table = desc.reset_index().drop('Variabile', axis = 1).to_markdown(index=False)

In [29]:
variables_listed = ' '.join(descriptions.loc[:, ['Variabile','Descrizione']].apply(lambda x: '\n -' +' : '.join(x.tolist()), axis = 1).values.tolist())

In [78]:
prompt = """
Sei un data scientist che lavora per una banca. La tabella compresa tra i tag <table> riporta il contributo, in termini di SHAP-value, di ogni variabile alla previsione di un tuo modello di classificazione.
Ogni variabile rappresenta una caratteristica del comportamento di un cliente.
Fornisci una spiegazione della previsione svolta dal modello cercando di dedurre il comportamento del cliente.
Concentrati sulle variabili che hanno influenzato maggiormente la previsione.

<table>
{table}
<table>
""".format(table = mkd_table)

In [79]:
print(prompt)


Sei un data scientist che lavora per una banca. La tabella compresa tra i tag <table> riporta il contributo, in termini di SHAP-value, di ogni variabile alla previsione di un tuo modello di classificazione.
Ogni variabile rappresenta una caratteristica del comportamento di un cliente.
Fornisci una spiegazione della previsione svolta dal modello cercando di dedurre il comportamento del cliente.
Concentrati sulle variabili che hanno influenzato maggiormente la previsione.

<table>
| Descrizione                                                                                                     |   SHAP_values |
|:----------------------------------------------------------------------------------------------------------------|--------------:|
| Somma Utilizzato su Somma Accordato negli ultimi due mesi                                                       |         0.396 |
| Media Giorni di Disponibilità CC negli ultimi tre mesi                                                          |    

In [80]:
risp = base_llm.invoke(prompt).content
print(risp)

Il modello ha effettuato la previsione basandosi principalmente su variabili legate all'uso del credito e al comportamento finanziario del cliente. 

La variabile con il maggiore SHAP-value positivo è "Somma Utilizzato su Somma Accordato negli ultimi due mesi" (0.396), indicando che il cliente ha utilizzato una grande parte del credito a sua disposizione negli ultimi due mesi. Questo potrebbe suggerire un possibile rischio di insolvenza, poiché l'uso eccessivo del credito può indicare problemi finanziari.

Altre variabili importanti con SHAP-value positivi sono "Media pesata per importo del massimo numero giorni di sconfino negli ultimi sei mesi" (0.257), "Flag Massimo Importo Sconfino > 100 nell'ultimo mese - cliente + responsabilità limitata" (0.207), "Somma Importo Effetti Trassati Insoluti su Importo Effetti Scaduti negli ultimi tre mesi" (0.322), "utilizzo su accordato revoca snb" (0.308) e "utilizzo su accordato autoliquidante snb" (0.267). Queste variabili indicano che il client

In [ ]:
Valuta contemporaneamente più variabili per dettagliare meglio l'operatività del cliente.


In [85]:
prompt = """
Sei un data scientist che lavora in una banca. Il tuo capo ha ricevuto la tabella compresa tra i tag <table> in cui è indicato il contributo ogni variabile alla previsione di un modello di classificazione (Il modello vuole prevedere il rischio di insolvenza di un prestito per un cliente).
Non essendo una persona tecnica, ha bisogno che lo aiuti ad interpretare la tabella. 
Le informazioni che gli interessano sapere sono:
- le caratteristiche principali del cliente
- come si sta comportando il cliente
- quale comportamento del cliente si può dedurre combinando il contributo di più variabili assieme 

Tieni a mente che un valore positivo di contributo indica che la variabile ha alzato il rischio di insolvenza; valori negativi indicano diminuzione del rischio mentre valori prossimi allo zero indicano scarsa influenza.
Focalizzati sulle variabili con contributo assoluto maggiore.

<table>
{table}
<table>
""".format(table = mkd_table)

print(prompt)


Sei un data scientist che lavora in una banca. Il tuo capo ha ricevuto la tabella compresa tra i tag <table> in cui è indicato il contributo ogni variabile alla previsione di un modello di classificazione (Il modello vuole prevedere il rischio di insolvenza di un prestito per un cliente).
Non essendo una persona tecnica, ha bisogno che lo aiuti ad interpretare la tabella. 
Le informazioni che gli interessano sapere sono:
- le caratteristiche principali del cliente
- come si sta comportando il cliente
- quale comportamento del cliente si può dedurre combinando il contributo di più variabili assieme 

Tieni a mente che un valore positivo di contributo indica che la variabile ha alzato il rischio di insolvenza; valori negativi indicano diminuzione del rischio mentre valori prossimi allo zero indicano scarsa influenza.
Focalizzati sulle variabili con contributo assoluto maggiore.

<table>
| Descrizione                                                                                        

In [86]:
risp = base_llm.invoke(prompt).content
print(risp)

Dalla tabella possiamo dedurre diverse informazioni sul cliente e sul suo comportamento finanziario.

Caratteristiche principali del cliente:
- Il cliente ha utilizzato una somma significativa rispetto a quella accordata negli ultimi due mesi, il che ha aumentato il rischio di insolvenza.
- Il cliente ha avuto una media di giorni di disponibilità del conto corrente negli ultimi tre mesi che ha contribuito positivamente al rischio di insolvenza.
- Il cliente ha avuto un numero massimo di giorni di sconfino (ovvero giorni in cui il saldo del conto è andato sotto zero) negli ultimi sei mesi che ha contribuito positivamente al rischio di insolvenza.
- Il cliente ha avuto un importo significativo di effetti protestati o richiamati rispetto agli effetti scaduti negli ultimi tre mesi, il che ha aumentato il rischio di insolvenza.

Comportamento del cliente:
- Il cliente ha avuto un utilizzo elevato rispetto all'accordato sia per la revoca snb che per l'autoliquidante snb, entrambi contribuend

In [92]:
prompt = """
Sei un data scientist che lavora in una banca. Il tuo capo ha ricevuto la tabella compresa tra i tag <table> in cui è indicato il contributo ogni variabile alla previsione di un modello di classificazione (Il modello vuole prevedere il rischio di insolvenza di un prestito per un cliente).
Non essendo una persona tecnica, ha bisogno che lo aiuti ad interpretare la tabella. 
Ragiona per passi: individua le caratteristiche principali del cliente considerato e come queste si relazionano al rischio di insolvenza. Successivamente, combina quando dedotto e descrivi come si sta comportando il cliente.
Concludi fornendo una spiegazione delle evidenze trovate giustificandole.

Focalizzati sulle variabili con contributo assoluto maggiore.

Tieni a mente che un valore positivo di contributo indica che la variabile ha alzato il rischio di insolvenza; valori negativi indicano diminuzione del rischio mentre valori prossimi allo zero indicano scarsa influenza.

<table>
{table}
<table>
""".format(table = mkd_table)


print(prompt)


Sei un data scientist che lavora in una banca. Il tuo capo ha ricevuto la tabella compresa tra i tag <table> in cui è indicato il contributo ogni variabile alla previsione di un modello di classificazione (Il modello vuole prevedere il rischio di insolvenza di un prestito per un cliente).
Non essendo una persona tecnica, ha bisogno che lo aiuti ad interpretare la tabella. 
Ragiona per passi: individua le caratteristiche principali del cliente considerato e come queste si relazionano al rischio di insolvenza. Successivamente, combina quando dedotto e descrivi come si sta comportando il cliente.
Concludi fornendo una spiegazione delle evidenze trovate giustificandole.

Focalizzati sulle variabili con contributo assoluto maggiore.

Tieni a mente che un valore positivo di contributo indica che la variabile ha alzato il rischio di insolvenza; valori negativi indicano diminuzione del rischio mentre valori prossimi allo zero indicano scarsa influenza.

<table>
| Descrizione                  

In [93]:
risp = base_llm.invoke(prompt).content
print(risp)

Dalla tabella possiamo dedurre diverse informazioni sul comportamento del cliente e sul suo rischio di insolvenza.

Innanzitutto, vediamo che la variabile con il contributo più alto è "Somma Utilizzato su Somma Accordato negli ultimi due mesi" con un valore di 0.396. Questo indica che il cliente ha utilizzato una grande parte del credito a sua disposizione negli ultimi due mesi, aumentando il rischio di insolvenza.

Un altro fattore che ha contribuito all'aumento del rischio è la "Media pesata per importo del massimo numero giorni di sconfino negli ultimi sei mesi" (0.257). Questo suggerisce che il cliente ha superato il limite del suo conto corrente per un numero significativo di giorni, un altro segnale di potenziale difficoltà finanziaria.

Inoltre, il "Flag Massimo Importo Sconfino > 100 nell'ultimo mese - cliente + responsabilità limitata" (0.207) indica che il cliente ha superato il suo limite di credito di un importo significativo nell'ultimo mese, un altro segnale di rischio.

